In [3]:
import numpy as np
from typing import List

In [4]:
def binary_encode(i: int, num_digits: int) -> List[int]:
    return np.array([i >> d & 1 for d in range(num_digits)])

In [55]:
for i in range(1, 10):
    print(binary_encode(i, 10))

[1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[1 0 1 0 0 0 0 0 0 0]
[0 1 1 0 0 0 0 0 0 0]
[1 1 1 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[1 0 0 1 0 0 0 0 0 0]


In [56]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else: return 0

def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [57]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(1)    # reproducible

In [58]:
NUM_DIGITS = 10
NUM_HIDDEN = 100
BATCH_SIZE = 32
X = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
y = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

X_train = X[100:]
y_train = y[100:]
X_valid = X[:100]
y_valid = y[:100]

In [59]:
X.shape, y.shape

((923, 10), (923,))

In [69]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

(torch.Size([823, 10]),
 torch.Size([823]),
 torch.Size([100, 10]),
 torch.Size([100]))

In [61]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_valid = torch.tensor(X_valid, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_valid = torch.tensor(y_valid, dtype=torch.int64)

In [81]:
class FizzBuzz(nn.Module):
    def __init__(self):
        super().__init__()
        self.first = nn.Linear(10, 100)
        self.relu = nn.ReLU()
        self.bactnorm1d = nn.BatchNorm1d(100)
        self.output = nn.Linear(100, 4)

    def forward(self, x):
        a = self.first(x)
        relu = self.relu(a)
        batcnorm = self.bactnorm1d(relu)
        return self.output(batcnorm)

In [82]:
net = FizzBuzz()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.03)

ds = TensorDataset(X_train, y_train)
loader = DataLoader(ds, batch_size=32, shuffle=True)

In [83]:
train_losses = []
test_losses = []

for _ in range(1000):
    running_loss = 0.0

    net.train()
    
    for i, (xx, yy) in enumerate(loader):
        y_pred = net(xx)
        loss = loss_fn(y_pred, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_losses.append(running_loss / i)
    
    net.eval()
    y_pred = net(X_valid)
    test_loss = loss_fn(y_pred, y_valid)
    print('Validation loss: ', test_loss, 'Training loss: ', loss)
    test_losses.append(test_loss.item())

Validation loss:  tensor(1.2031, grad_fn=<NllLossBackward0>) Training loss:  tensor(1.2707, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(1.1365, grad_fn=<NllLossBackward0>) Training loss:  tensor(1.2441, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(1.1811, grad_fn=<NllLossBackward0>) Training loss:  tensor(1.1383, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(1.2882, grad_fn=<NllLossBackward0>) Training loss:  tensor(1.2332, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(1.3296, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.9795, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(1.3944, grad_fn=<NllLossBackward0>) Training loss:  tensor(1.3072, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(1.4020, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.9569, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(1.3477, grad_fn=<NllLossBackward0>) Training loss:  tensor(1.2010, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(1.3668, grad_fn

Validation loss:  tensor(0.5902, grad_fn=<NllLossBackward0>) Training loss:  tensor(1.2726, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.6060, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.7573, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.6307, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4518, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5956, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.8565, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.7028, grad_fn=<NllLossBackward0>) Training loss:  tensor(1.0963, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.7622, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.6372, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.6502, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4197, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.6274, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.5813, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.7361, grad_fn

Validation loss:  tensor(0.5144, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2758, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5752, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3011, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.6034, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4684, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5779, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.9488, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5031, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4797, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.6312, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3144, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4292, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3068, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5148, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3965, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4993, grad_fn

Validation loss:  tensor(0.3216, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4833, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3352, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1601, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3163, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2166, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.2541, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2573, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.2535, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.0512, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4003, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2245, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.1906, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4150, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.2638, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.5953, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5779, grad_fn

Validation loss:  tensor(0.4302, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1827, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.2598, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2739, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4199, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3722, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3591, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4552, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.1726, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.5096, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4679, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.5668, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3529, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3222, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.1988, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1811, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.2877, grad_fn

...

Validation loss:  tensor(0.3744, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2086, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4432, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.5405, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3445, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4130, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3409, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.5960, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.1896, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.6502, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3264, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2545, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.1732, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1638, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3113, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.0662, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3836, grad_fn

Validation loss:  tensor(0.5508, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2538, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.2718, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.0656, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3854, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.6012, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4757, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2536, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3112, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3407, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5048, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4146, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3810, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1265, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3979, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2269, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5829, grad_fn

Validation loss:  tensor(0.2967, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2520, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3600, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3800, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4414, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.0788, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4015, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.5728, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.2817, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1854, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3493, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2403, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4198, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2624, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4655, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1637, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3205, grad_fn

Validation loss:  tensor(0.5535, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2975, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3868, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3997, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5936, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.0984, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.8789, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4995, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4285, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2273, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4875, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.2993, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5277, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3733, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4835, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1685, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.3974, grad_fn

Validation loss:  tensor(0.5965, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3652, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4688, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.3819, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4364, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.4983, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4366, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.6128, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.8965, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1465, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4796, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1998, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5335, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.5478, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.4044, grad_fn=<NllLossBackward0>) Training loss:  tensor(0.1556, grad_fn=<NllLossBackward0>)
Validation loss:  tensor(0.5821, grad_fn

In [84]:
numbers = np.arange(1, 101)
X_test = np.array([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
X_test = torch.tensor(X_test, dtype=torch.float32)

net.eval()
_, y_pred = torch.max(net(X_test), 1)

output = np.vectorize(fizz_buzz)(numbers, y_pred)
print(output)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' '12' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' '21' '22' '23' 'fizz'
 'fizzbuzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' '42' '43' '44' 'fizzbuzz' '46' '47'
 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz'
 '71' 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' '81' '82'
 '83' 'fizz' 'buzz' '86' '87' '88' '89' 'fizzbuzz' '91' '92' '93' '94'
 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']
